In [ ]:
import numpy as np
import itertools
import os

import dotenv
dotenv.load_dotenv()

import sys, os
dir_head = os.environ.get("REPO_DIR", ".")
sys.path.insert(0, os.path.join(dir_head, "src"))

import numpy as np
from pixell import enmap

import warnings
import itertools

warnings.filterwarnings("ignore")

# files from the src dir in git
import utils as ut
import covariance as cov

import matplotlib.pyplot as plt
import glob

cov_dir = os.environ['COV_DIR']

In [ ]:
config_data_fname = "../configs/config_mcmc.yaml"

try:
    cf = ut.get_config_file(config_data_fname)
except FileNotFoundError:
    print(f"Config file not found: {config_data_fname}")

cluster_region = ut.get_region(region_center_ra=cf['region_center_ra'], 
                                region_center_dec=cf['region_center_dec'],
                                region_width=cf['region_width'])

data_list, beam_list, data_wcs_list = [], [], []

print("Appending data.")
for _, data in enumerate(cf['data']):
    freq = data.split('_')[0]
    array = data.split('_')[1]
    inst = data.split('_')[2]
    scan = data.split('_')[3]

    if inst == 'act':
        if scan == 'dr6v2':
            data_dir = os.getenv("ACT_DATADIR")
        elif scan == 'dr6v4':
            data_dir = cf['act_data_dir_dr6v4']
        else: 
            raise ValueError("Invalid scan type")

    elif inst == 'planck':
        data_dir = os.getenv("PLANCK_DATADIR")
    
    else: 
        raise ValueError("Undefined instrument.")

    if cf['system_type'] == 'sim':
        data_str_flag = "_srcfree_model"
    elif cf['system_type'] == 'real':
        data_str_flag = "_srcfree"
    elif cf['system_type'] == 'real_with_sources':
        data_str_flag = ""
    else:
        raise ValueError("Undefined system type.")

    str_data = f"{data_dir}/*{array}*{freq}*coadd*map{data_str_flag}.fits"

    data_coadd = np.sort(glob.glob(str_data))[0]
    data_coadd = ut.imap_dim_check(enmap.read_map(data_coadd, box=cluster_region))
    
    # Convert data from uK to Jy/sr
    ff = ut.flux_factor(array, str(freq)) 
    data_coadd *= ff     
    data_wcs = data_coadd.wcs
    data_shape = data_coadd.shape

    beam_tmp = ut.get_2d_beam(freq=freq, 
                                array=array, 
                                inst=inst, 
                                data_shape=data_shape, 
                                version=str(scan),
                                data_wcs=data_wcs)
    
    apod_mask = (enmap.apod(data_coadd*0+1, cf['apod_pix']))

    data_list.append(np.fft.fft2(data_coadd))
    beam_list.append(np.array(beam_tmp))

    data_wcs_list.append(data_wcs) 

data_wcs = data_wcs_list[0]
mean_apod_mask = np.mean(apod_mask)

# Covariance appender
print("Appending covariance.")
combos = []
for combo in itertools.product(cf['data'], cf['data']):
    combos.append(combo)

covar_list = []
covar_ref_list = []
combo_names = []

for _, combo in enumerate(combos):
    # print(f"Combo: {combo}")
    freq1 = combo[0].split('_')[0]
    array1 = combo[0].split('_')[1]
    inst1 = combo[0].split('_')[2]
    scan1 = combo[0].split('_')[3]

    freq2 = combo[1].split('_')[0]
    array2 = combo[1].split('_')[1]
    inst2 = combo[1].split('_')[2]
    scan2 = combo[1].split('_')[3]

    tpsd = np.load(f"{cov_dir}/cov_{freq1}_{array1}_{inst1}_{scan1}_{freq2}_{array2}_{inst2}_{scan2}.npy")
    # tpsd_ref = np.load(f"{cov_dir}_ref/cov_{freq2}_{array2}_{inst2}_{scan2}_{freq1}_{array1}_{inst1}_{scan1}.npy")
    
    covar_list.append(tpsd.ravel())
    # covar_ref_list.append(tpsd_ref.ravel())
    combo_names.append(combo)

print(combo_names)
data_list_array = np.array(data_list)
beam_list_array = np.array(beam_list)

npix = len(data_list_array[0].ravel())
nmaps = len(cf['data'])

icov = np.linalg.inv(np.array(covar_list).T.reshape(npix, nmaps, nmaps))
# icov_ref = np.linalg.inv(np.array(covar_ref_list).T.reshape(npix, nmaps, nmaps))


In [ ]:
def get_sim_cov(fname):
    # Simulated power spectrum
    system_sim_real = enmap.read_map(fname)
    system_sim_fft  = np.fft.fft2(system_sim_real)
    system_sim_cov  = system_sim_fft * system_sim_fft.conj()
    system_sim_cov  = enmap.ndmap(system_sim_cov, system_sim_real.wcs)
    return system_sim_cov

In [ ]:
binsize = 200
plt.figure(figsize=(10,10), dpi=100)  
# colors = plt.cm.viridis(np.linspace(0, 1, len(combo_names)))
colors = np.random.rand(len(combo_names), 3)
# for i, (combo, covmat, covmat_ref) in enumerate(zip(combo_names, covar_list, covar_ref_list)):
for i, (combo, covmat) in enumerate(zip(combo_names, covar_list)):
    if 'planck' in combo[0] or 'planck' in combo[1]:
        continue
    # auto only
    covmat = enmap.ndmap(covmat.reshape(data_list_array[0].shape), data_wcs)
    # covmat_ref = enmap.ndmap(covmat_ref.reshape(data_list_array[0].shape), data_wcs)
    if combo[0] == combo[1]:
        # if 'planck' in combo[0]:  ls = '--'
        # else:  ls = '-'
        # if 'pa6' in combo[0] or 'pa6' in combo[1]:
        #     continue
        # if '353' in combo[0] or '353' in combo[1]:
        #     continue
        # if '217' in combo[0] or '217' in combo[1]:
        #     continue
        # if '545' in combo[0] or '545' in combo[1]:
        #     continue
        # if '143' in combo[0] or '143' in combo[1]:
        #     continue
        # if '150' in combo[0] or '150' in combo[1]:
        #     continue
        b_tpsd, l_tpsd = enmap.lbin(map=np.abs(covmat), bsize=binsize)
        # b_tpsd_ref, l_tpsd_ref = enmap.lbin(map=np.abs(covmat_ref), bsize=binsize)
        # plt.loglog(l_tpsd, b_tpsd, label=f"{combo[0]}-{combo[1]}", ls=ls)
        # plt.loglog(l_tpsd_ref, b_tpsd_ref, label=f"{combo[0]}-{combo[1]} (ref)", ls=ls)
        plt.loglog(l_tpsd, b_tpsd, label=f"{combo[0]}-{combo[1]}", ls='-', c=colors[i])
        # plt.loglog(l_tpsd_ref, b_tpsd_ref, label=f"{combo[0]}-{combo[1]} (ref)", ls='--', c=colors[i])

show_sim = False
if show_sim:
    cov_c1_c2 = get_sim_cov(f"../data/system_sim_pa5_98_c1_c2.fits")
    b_tpsd, l_tpsd = enmap.lbin(map=np.abs(cov_c1_c2), bsize=binsize)
    plt.loglog(l_tpsd, b_tpsd, label='Simulated (c1+c2)', ls='-')

    cov_fil = get_sim_cov(f"../data/system_sim_pa5_98_fil.fits")
    b_tpsd, l_tpsd = enmap.lbin(map=np.abs(cov_fil), bsize=binsize)
    plt.loglog(l_tpsd, b_tpsd, label='Simulated (fil)', ls='-')

    cov_c1_c2_fil = get_sim_cov(f"../data/system_sim_pa5_98_c1_c2_fil.fits")
    b_tpsd, l_tpsd = enmap.lbin(map=np.abs(cov_c1_c2_fil), bsize=binsize)
    plt.loglog(l_tpsd, b_tpsd, label='Simulated (c1+c2+fil)', ls='-')

# Adjust the main plot to make room for the legend
plt.subplots_adjust(bottom=0.2)

# Place legend outside the plot, below it, with 3 columns
plt.legend(loc='upper center', fontsize=12, bbox_to_anchor=(0.5, -0.1), ncol=2, fancybox=True, shadow=True)
plt.tight_layout()  
plt.xlabel('l')
plt.ylabel('Power Spectrum')


In [ ]:
binsize = 200
fig, ax1 = plt.subplots(figsize=(10,6), dpi=200)  

for combo, covmat in zip(combo_names, covar_list):
    # auto only
    covmat = enmap.ndmap(covmat.reshape(data_list_array[0].shape), data_wcs)
    if combo[0] == combo[1]:
        if 'planck' in combo[0]:
            ls = '--'
        else:
            ls = '-'
        if 'pa6' in combo[0] or 'pa6' in combo[1]:
            continue
        if '353' in combo[0] or '353' in combo[1]:
            continue
        if '217' in combo[0] or '217' in combo[1]:
            continue
        if '545' in combo[0] or '545' in combo[1]:
            continue
        if '143' in combo[0] or '143' in combo[1]:
            continue
        if '150' in combo[0] or '150' in combo[1]:
            continue
        b_tpsd, l_tpsd = enmap.lbin(map=np.abs(covmat), bsize=binsize)
        ax1.loglog(l_tpsd, b_tpsd, label=f"{combo[0]}-{combo[1]}", ls=ls)

show_sim = True
if show_sim:
    cov_c1_c2 = get_sim_cov(f"../data/system_sim_pa5_98_c1_c2.fits")
    b_tpsd, l_tpsd = enmap.lbin(map=np.abs(cov_c1_c2), bsize=binsize)
    ax1.loglog(l_tpsd, b_tpsd, label='Simulated (c1+c2)', ls='-')

    cov_fil = get_sim_cov(f"../data/system_sim_pa5_98_fil.fits")
    b_tpsd, l_tpsd = enmap.lbin(map=np.abs(cov_fil), bsize=binsize)
    ax1.loglog(l_tpsd, b_tpsd, label='Simulated (fil)', ls='-')

    cov_c1_c2_fil = get_sim_cov(f"../data/system_sim_pa5_98_c1_c2_fil.fits")
    b_tpsd, l_tpsd = enmap.lbin(map=np.abs(cov_c1_c2_fil), bsize=binsize)
    ax1.loglog(l_tpsd, b_tpsd, label='Simulated (c1+c2+fil)', ls='-')

# Create a secondary x-axis for degrees
ax2 = ax1.twiny()
# Get the current x-axis limits (in multipole l)
l_min, l_max = ax1.get_xlim()

# Convert l to degrees: θ(degrees) = 180°/l
def l_to_deg(l):
    return 180/l

# Set the secondary axis limits and scale
ax2.set_xscale('log')
ax2.set_xlim(l_to_deg(l_max), l_to_deg(l_min))  # Note: reversed because degrees decrease as l increases
ax2.set_xlabel('Angular Scale [deg]')

# Adjust the main plot to make room for both axes
plt.subplots_adjust(bottom=0.2, top=0.85)

# Place legend outside the plot, below it, with 3 columns
plt.legend(loc='lower left', fontsize=12)

ax1.set_xlabel('Multipole l')
ax1.set_ylabel('Power Spectrum')
plt.tight_layout()

In [ ]:
# show the SNR for a given combination
sim_target = 'c1_c2_fil'
# sim_target = 'fil'

binsize = 200
fig, ax1 = plt.subplots(figsize=(10,6), dpi=200)  

def get_name(combo):
    combo1 = ' '.join(combo[0].split('_')[:3])
    combo2 = ' '.join(combo[1].split('_')[:3])
    return f"{combo1} -- {combo2}"

for combo, covmat in zip(combo_names, covar_list):
    # auto only
    covmat = enmap.ndmap(covmat.reshape(data_list_array[0].shape), data_wcs)
    if combo[0] == combo[1]:
        if 'pa6' in combo[0] or 'pa6' in combo[1]:
            continue
        if '353' in combo[0] or '353' in combo[1]:
            continue
        if '217' in combo[0] or '217' in combo[1]:
            continue
        if '545' in combo[0] or '545' in combo[1]:
            continue
        if '143' in combo[0] or '143' in combo[1]:
            continue
        if '150' in combo[0] or '150' in combo[1]:
            continue
        cov_sim = get_sim_cov({
            'c1_c2': f"../data/system_sim_pa5_98_c1_c2.fits",
            'fil': f"../data/system_sim_pa5_98_fil.fits",
            'c1_c2_fil': f"../data/system_sim_pa5_98_c1_c2_fil.fits"
        }[sim_target])
        b_tpsd, l_tpsd = enmap.lbin(map=np.abs(covmat), bsize=binsize)
        b_tpsd_sim, l_tpsd_sim = enmap.lbin(map=np.abs(cov_c1_c2), bsize=binsize)
        snr = b_tpsd_sim / b_tpsd
        ax1.loglog(l_tpsd, snr, label=f"{get_name(combo)} SNR ({sim_target})", ls='-')

# Place legend outside the plot, below it, with 3 columns
plt.legend(loc='lower left', fontsize=12)
plt.xlim(right=10000)
plt.ylim(bottom=1e-6)


# Create a secondary x-axis for degrees
ax2 = ax1.twiny()
# Get the current x-axis limits (in multipole l)
l_min, l_max = ax1.get_xlim()

# Convert l to degrees: θ(degrees) = 180°/l
def l_to_deg(l):
    return 180/l

# Set the secondary axis limits and scale
ax2.set_xscale('log')
ax2.set_xlim(l_to_deg(l_max), l_to_deg(l_min))  # Note: reversed because degrees decrease as l increases
ax2.set_xlabel('Angular Scale [deg]')

# Adjust the main plot to make room for both axes
plt.subplots_adjust(bottom=0.2, top=0.85)


ax1.set_xlabel(r'$\ell$')
ax1.set_ylabel('SNR')
plt.tight_layout()

In [ ]:
binsize = 200

for combo, covmat in zip(combo_names, covar_list):
    # auto only
    covmat = enmap.ndmap(covmat.reshape(data_list_array[0].shape), data_wcs)
    if combo[0] == combo[1]:
        fig, ax1 = plt.subplots(figsize=(8,6), dpi=150)  
        if 'planck' in combo[0]:
            ls = '--'
        else:
            ls = '-'
        b_tpsd, l_tpsd = enmap.lbin(map=np.abs(covmat), bsize=binsize)
        ax1.loglog(l_tpsd, b_tpsd, label=f"{combo[0]}-{combo[1]}", ls=ls)
        freq, array, inst = combo[0].split('_')[0], combo[0].split('_')[1], combo[0].split('_')[2]
        
        show_sim = True
        if show_sim:
            cov_c1_c2 = get_sim_cov(f"../data/system_sim_{array}_{freq}_c1_c2.fits")
            b_tpsd, l_tpsd = enmap.lbin(map=np.abs(cov_c1_c2), bsize=binsize)
            ax1.loglog(l_tpsd, b_tpsd, label=f'Simulated ({array} {freq}GHz c1+c2)', ls='-')

            cov_fil = get_sim_cov(f"../data/system_sim_{array}_{freq}_fil.fits")
            b_tpsd, l_tpsd = enmap.lbin(map=np.abs(cov_fil), bsize=binsize)
            ax1.loglog(l_tpsd, b_tpsd, label=f'Simulated ({array} {freq}GHz fil)', ls='-')

            cov_c1_c2_fil = get_sim_cov(f"../data/system_sim_{array}_{freq}_c1_c2_fil.fits")
            b_tpsd, l_tpsd = enmap.lbin(map=np.abs(cov_c1_c2_fil), bsize=binsize)
            ax1.loglog(l_tpsd, b_tpsd, label=f'Simulated ({array} {freq}GHz c1+c2+fil)', ls='-')
        # Create a secondary x-axis for degrees
        ax2 = ax1.twiny()
        # Get the current x-axis limits (in multipole l)
        l_min, l_max = ax1.get_xlim()

        # Convert l to degrees: θ(degrees) = 180°/l
        def l_to_deg(l):
            return 180/l

        # Set the secondary axis limits and scale
        ax2.set_xscale('log')
        ax2.set_xlim(l_to_deg(l_max), l_to_deg(l_min))  # Note: reversed because degrees decrease as l increases
        ax2.set_xlabel('Angular Scale [deg]')

        # Adjust the main plot to make room for both axes
        # plt.subplots_adjust(bottom=0.2, top=0.85)

        # Place legend outside the plot, below it, with 3 columns
        ax1.legend(loc='lower left', fontsize=8)

        ax1.set_xlabel('Multipole l')
        ax1.set_ylabel('Power Spectrum')
        plt.tight_layout()
        plt.title(f"{array} {freq} GHz")
        plt.show()



In [ ]:
ut.plot_image(np.abs(covar_list[0].reshape(300, 300)), interval_type='zscale')

In [ ]:
def check_complex_covariance(matrix, tol=1e-8):
    # Check if the matrix is square
    if matrix.shape[0] != matrix.shape[1]:
        return False, "Matrix is not square"

    # Check if the matrix is Hermitian
    if not np.allclose(matrix, np.conjugate(matrix.T), atol=tol):
        return False, "Matrix is not Hermitian"

    # Check if the matrix is positive semidefinite
    eigenvalues = np.linalg.eigvals(matrix)
    
    # print the eigenvalues
    #print("Eigenvalues:", np.real(eigenvalues))

    # Identify negative eigenvalues (below the tolerance threshold)
    negative_eigenvalues = np.real(eigenvalues)[np.real(eigenvalues) < -tol]
    
    if len(negative_eigenvalues) > 0:
        print("Negative eigenvalues:", negative_eigenvalues)
        return False, "Matrix is not positive semidefinite"

    return True, "Matrix is a valid complex covariance matrix"

# icov.shape: npix x nmap x nmap

covmat = np.array(covar_list).T.reshape(npix, nmaps, nmaps)
for idx in range(npix):
    cov_idx = np.array(covmat[idx, :, :])

    is_valid_complex, message = check_complex_covariance(cov_idx)

    if not is_valid_complex:
        print(f"Complex check fail: Index: {idx}")
        print(message)

In [ ]:
import matplotlib.pyplot as plt

# Assuming icov is a list or array of covariance matrices
num_plots = 60 #len(icov)
# mid = len(icov)//2
# start = mid - num_plots//2
# end = mid + num_plots//2

# Calculate the number of rows and columns for the subplot grid
num_cols = int(np.ceil(np.sqrt(num_plots)))
num_rows = int(np.ceil(num_plots / num_cols))

# Create a large figure
fig, axes = plt.subplots(num_rows, num_cols, figsize=(1*num_cols, 1*num_rows))

# Flatten the axes array for easy iteration
axes = axes.flatten()
# Loop over i and plot each image
for i in range(num_plots):
    ax = axes[i]
    im = ax.imshow(np.abs(icov[i + (i*data_shape[0])]))
    modlmap = enmap.modlmap(data_shape, data_wcs)
    l = modlmap.ravel()[i + (i*data_shape[0])]
    ax.set_title(f"l = {l:.0f}", fontsize=6)
    ax.axis('off')

# Remove any extra subplots
for i in range(num_plots, len(axes)):
    fig.delaxes(axes[i])

# Adjust the layout to remove gaps
plt.subplots_adjust(wspace=0.3, hspace=0.3)

# Display the plot
plt.show()

In [ ]:
for combo, covmat in zip(combo_names, covar_list):
    if (combo[0] != combo[1]): continue
    # auto only
    covmat = enmap.ndmap(covmat.reshape(data_list_array[0].shape), data_wcs)
    # ut.plot_image(np.abs(np.fft.fftshift(covmat)), interval_type='zscale', title=f"{combo[0]}-{combo[1]}")
    ut.plot_image(np.abs(np.fft.fftshift(covmat)), interval_type='zscale', title=f"{combo[0]}-{combo[1]}")